In [ ]:
!python -m spacy download en_core_web_trf


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import warnings
warnings.filterwarnings("ignore")
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm

In [10]:
df_train=pd.read_csv('final_train.csv', encoding='utf-8', encoding_errors='ignore')
df_train.columns=['sentence','tags']

df_test=pd.read_csv('final_test.csv', encoding='utf-8', encoding_errors='ignore')
df_test.columns=['sentence','tags']

In [11]:
# function to obtain entity format according to Spacy
def get_entity_tuple(df):
  list_of_entities = []
  for x in range(len(df)):
    d = {"entities": []}
    i, j = 0, 0
    idx = -1
    
    tokens = df["tags"][x].split()
    words = df['sentence'][x].split()
    if len(words) != len(tokens):
      continue

    temp = df["sentence"][x] + " "
    for char in temp:
      if char == " ":
        d["entities"].append((i, j, tokens[idx]))
        j += 1
        i = j
        idx += 1
      else:
        j += 1
    list_of_entities.append((df["sentence"][x], d))
  
  return list_of_entities

In [12]:
# apply on train and test set
train_entities = get_entity_tuple(df_train)
test_entities = get_entity_tuple(df_test)

In [13]:
db = DocBin()
nlp = spacy.load("en_core_web_trf")

for text, annot in tqdm(train_entities): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./train.spacy") # save the docbin object


100%|██████████| 6611/6611 [00:00<00:00, 7190.88it/s]


Skipping entity


In [14]:
db = DocBin()
nlp = spacy.load("en_core_web_trf")

for text, annot in tqdm(test_entities): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./test.spacy") # save the docbin object

100%|██████████| 2187/2187 [00:00<00:00, 6283.87it/s]

Skipping entity


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy --gpu-id 0

2023-03-13 16:39:44.539134: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-13 16:39:44.539464: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-13 16:39:44.539513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-03-13 16:39:59,799] [INFO] Set